In [55]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()


In [56]:
df = pd.read_sql(
    """
    select ticker, date, ret, roeq, bm, siccd
    from data
    order by ticker, date
    """, 
    conn
)

df = df.dropna()
df["actual"] = df.ret

In [57]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)

features = ["roeq", "bm"]

df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d: 
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )     
)

In [58]:
inds = pd.read_csv(
  "docs/files/siccodes12.csv", 
  index_col="industry"
)

In [60]:
def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

In [61]:
codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes.name = "industry" 
codes.index.name = "siccd"
df = df.merge(codes, on="siccd")

In [63]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
model = LinearRegression()

transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)
transform2 = PolynomialFeatures(degree=2)
pipe = make_pipeline(
    transform1,
    transform2,
    model
)

In [64]:
features.append("industry")

In [66]:
df = df.set_index(["date", "ticker"])
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None
for train_date, end_date in zip(dates[:-1], dates[1:]):
  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]
  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  pipe.fit(Xtrain, ytrain)
  pred = pipe.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
df["predict"] = predictions

In [67]:
numlong = 100
numshort = 100

df["rank_from_top"] = df.groupby("date").predict.rank(
  method="first", 
  ascending=False
)
df["long"] = df.rank_from_top <= numlong

df["rank_from_bottom"] = df.groupby("date").predict.rank(
  method="first"
)
df["short"] = df.rank_from_bottom <= numshort

In [69]:
df = df[df.index.get_level_values("date") >= dates[0]]

long_rets = df.groupby("date").apply(
    lambda d: (d.long*d.ret).mean()
)
short_rets = df.groupby("date").apply(
    lambda d: (d.short*d.ret).mean()
)

In [70]:
short_rets

date
2005-01   -0.013855
2005-02   -0.011193
2005-03   -0.010480
2005-04   -0.007949
2005-05    0.000693
             ...   
2021-11   -0.022028
2021-12   -0.019210
2022-01   -0.007699
2022-02    0.002348
2022-03    0.006857
Length: 207, dtype: float64

In [75]:
predictions = {
    "AAPL": .1,
    "TSLA": 0,
    "NVDA": 0.2
}

tradable = {
    "AAPL": True,
    "TSLA": False,
    "BUD": False,
    "MSFT": True
}

{x: predictions[x] for x in predictions if x in tradable and tradable[x]}

{'AAPL': 0.1}

In [77]:
t = pd.DataFrame(pd.Series(predictions))
t

,0
AAPL,0.1
TSLA,0.0
NVDA,0.2


In [81]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer
from joblib import load

from sqlalchemy import create_engine
import pymssql
from datetime import datetime

from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKXP6PHRQXOD9Y7ZQZLR"
SECRET_KEY = "1b1gHrYXJbi3WPXaEbeq72WYdFs9r9ODZ04RvMF7"
numstocks = 200

qt = QuantileTransformer(output_distribution="normal")
pipe = load("../2023-722-binder/files/linear_model_2023-01-20.joblib")

c:\Users\keb7\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator PolynomialFeatures from version 1.1.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\keb7\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\keb7\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator Pipeline from version 1.1.2 when using 

In [83]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
model = LinearRegression()

transform1 = make_column_transformer(
    (OneHotEncoder(), ["industry"]),
    remainder="passthrough"
)
transform2 = PolynomialFeatures(degree=2)
pipe = make_pipeline(
    transform1,
    transform2,
    model
)



server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select ticker, date, ret, roeq, bm, siccd
    from data
    order by ticker, date
    """, 
    conn
)

df = df.dropna()
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)

features = ["roeq", "bm"]

df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d: 
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )     
)



FileNotFoundError: [Errno 2] No such file or directory: 'files/siccodes12.csv'

In [84]:
inds = pd.read_csv(
  "docs/files/siccodes12.csv", 
  index_col="industry"
)

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes.name = "industry" 
codes.index.name = "siccd"
df = df.merge(codes, on="siccd")
features.append("industry")

df = df.set_index(["date", "ticker"])
pipe.fit(df[features], df.ret)



Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['industry'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('linearregression', LinearRegression())])

In [85]:
d = df[df.index.get_level_values("date")=="2022-03"]
d = d.reset_index().drop(columns=["date"]).set_index("ticker")

In [94]:
d["predict"] = pipe.predict(d[features])

In [86]:
from alpaca.trading.client import TradingClient
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.enums import OrderSide, TimeInForce, AssetClass

KEY = "PKXP6PHRQXOD9Y7ZQZLR"
SECRET_KEY = "1b1gHrYXJbi3WPXaEbeq72WYdFs9r9ODZ04RvMF7"

In [87]:
from alpaca.trading.client import TradingClient

trading_client = TradingClient(KEY, SECRET_KEY, paper=True)
account = trading_client.get_account()

equity = float(account.equity)

In [96]:
assets = trading_client.get_all_assets()

d["tradable"] = {
  x.symbol: x.tradable for x in assets
}

d["shortable"] = {
  x.symbol: x.shortable for x in assets
}

In [100]:
data_client = StockHistoricalDataClient(KEY, SECRET_KEY)
params = StockLatestQuoteRequest(
  symbol_or_symbols=d.index.to_list()
)
quotes = data_client.get_stock_latest_quote(params)

In [101]:
df = d

In [104]:
df["ask"] = {x: quotes[x].ask_price for x in quotes}
df["bid"] = {x: quotes[x].bid_price for x in quotes}